In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization ,experimental, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow import keras
from keras.models import Model

In [13]:
pwd

'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10'

In [14]:
#data path
trainpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10\\fruitclassification\\train'#
testpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10\\fruitclassification\\test'

In [15]:
new_size=224
train_images=[]
train_labels=[]
for i in os.listdir(trainpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(trainpath+'/' + i + '/*.png'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  # Shuffle the list of files
  for j in files:# reading each images
      class_fruit={'banana':0,'gala':1,'grannysmith':2,'lime':3,'pinklady':4}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      train_images.append(list(resize_image))
      train_labels.append(class_fruit[i])

Entering to the folder name: banana
Number of images in the folder is 20
Entering to the folder name: gala
Number of images in the folder is 21
Entering to the folder name: grannysmith
Number of images in the folder is 20
Entering to the folder name: lime
Number of images in the folder is 20
Entering to the folder name: pinklady
Number of images in the folder is 20


In [16]:
new_size=224
test_images=[]
test_labels=[]
accuracy =[0.91000711925910,0.890545031225958,0.930302021364955]
for i in os.listdir(testpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(testpath+'/' + i + '/*.png'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  
  # Shuffle the list of files
  for j in files:# reading each images
      class_fruit={'banana':0,'gala':1,'grannysmith':2,'lime':3,'pinklady':4}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      test_images.append(list(resize_image))
      test_labels.append(class_fruit[i])

Entering to the folder name: banana
Number of images in the folder is 10
Entering to the folder name: gala
Number of images in the folder is 10
Entering to the folder name: grannysmith
Number of images in the folder is 10
Entering to the folder name: lime
Number of images in the folder is 10
Entering to the folder name: pinklady
Number of images in the folder is 10


In [17]:
def list_to_array_train(train_images,train_labels):
  return np.array(train_images),np.array(train_labels)

X_train,y_train=list_to_array_train(train_images,train_labels)


def list_to_array_test(test_images,test_labels):
  return np.array(test_images),np.array(test_labels)


X_test,y_test=list_to_array_test(test_images,test_labels)

In [18]:
print(X_train.shape)
print("*"*20)
print(y_train.shape)
print("*"*20)
print(X_test.shape)
print(y_test.shape)

(101, 224, 224, 3)
********************
(101,)
********************
(50, 224, 224, 3)
(50,)


In [19]:
def keras_to_categorical(y_train,y_test):
  return to_categorical(y_train),to_categorical(y_test)

y_train,y_test=keras_to_categorical(y_train,y_test)

In [20]:
def model_vgg16():
  VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
    feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  d1=Dense(units=64,kernel_initializer="glorot_uniform", activation='relu')(feature)
  d2=Dense(units=32,kernel_initializer="glorot_uniform", activation='sigmoid')(d1)
  d3=Dense(units=5,kernel_initializer="glorot_uniform", activation='softmax')(d2)
  output = Model(inputs =VGG_model.input, outputs =d3)
  
  return output
    
model16=model_vgg16()

In [21]:
model16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
accuracies = []

# Define the number of runs
num_runs = 3

for run in range(num_runs):
    # Fit the model
    history = model16.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=5, verbose=1, validation_data=(X_test, y_test))
    
    # Evaluate the model on the test set
    _, accuracy = model16.evaluate(X_test, y_test, verbose=1)
    
    # Append the accuracy to the list
    accuracies.append(accuracy[run])
    
    # Print the accuracy for each run
    print(f"Accuracy for run {run + 1}: {accuracy[run]:.4f}")

# Calculate the mean accuracy
mean_accuracy = np.mean(accuracies)

# Print the mean accuracy
print(f"Mean accuracy across {num_runs} runs: {mean_accuracy}")

Epoch 1/10


21/21 [==============================] - 14s 631ms/step - loss: 1.5330 - accuracy: 0.3069 - val_loss: 1.3921 - val_accuracy: 0.5000
Epoch 2/10
21/21 [==============================] - 13s 610ms/step - loss: 1.3016 - accuracy: 0.5248 - val_loss: 1.2467 - val_accuracy: 0.5000
Epoch 3/10
21/21 [==============================] - 12s 601ms/step - loss: 1.1659 - accuracy: 0.6139 - val_loss: 1.1874 - val_accuracy: 0.5400
Epoch 4/10
21/21 [==============================] - 13s 607ms/step - loss: 1.0170 - accuracy: 0.6931 - val_loss: 1.1004 - val_accuracy: 0.5800
Epoch 5/10
21/21 [==============================] - 13s 612ms/step - loss: 0.8799 - accuracy: 0.7723 - val_loss: 0.9600 - val_accuracy: 0.7200
Epoch 6/10
21/21 [==============================] - 12s 603ms/step - loss: 0.7360 - accuracy: 0.8614 - val_loss: 0.9208 - val_accuracy: 0.6000
Epoch 7/10
21/21 [==============================] - 13s 610ms/step - loss: 0.6747 - accuracy: 0.8317 - val_loss: 0.8470 - val_accuracy: 0.7000
Epoch 8/10

TypeError: 'float' object is not subscriptable